## AndLink and EvaluationLink examples

In [1]:
import common
from opencog.utilities import initialize_opencog
from opencog.atomspace import AtomSpace, types
from opencog.type_constructors import *
from opencog.bindlink import execute_atom

create atomspace and set it as default

In [2]:
atomspace = AtomSpace()
initialize_opencog(atomspace)

### knowledge base

Color and size are attributes.
PredicateNode is used to define relations between different objects.

In [3]:
InheritanceLink(PredicateNode("size"), ConceptNode("attribute"))
InheritanceLink(PredicateNode("color"), ConceptNode("attribute"))

different types of color and size, InheritanceLink is similiar to verb "is" like it is used in natural languages. 

In [4]:
InheritanceLink(PredicateNode("small"), ConceptNode("size"))
InheritanceLink(PredicateNode("big"), ConceptNode("size"))
InheritanceLink(PredicateNode("green"), ConceptNode("color"))
InheritanceLink(PredicateNode("red"), ConceptNode("color"))
InheritanceLink(PredicateNode("black"), ConceptNode("color"))

(InheritanceLink
  (PredicateNode "black") ; [3641749076221326293][1]
  (ConceptNode "color") ; [4437742730051705013][1]
) ; [17115093303551131577][1]

animals which have some attributes defined using EvaluationLink  
EvaluationLink defines predicate for particular objects.

In [5]:
def assign_property(concept, relation, tv):
    EvaluationLink(relation,
                   concept).tv = tv

assign_property(ConceptNode("frog"), PredicateNode("small"), TruthValue(0.8, 0.9))
assign_property(ConceptNode("frog"), PredicateNode("green"), TruthValue(0.7, 0.9))
assign_property(ConceptNode("tiger"), PredicateNode("big"), TruthValue(0.9, 0.9))
assign_property(ConceptNode("tiger"), PredicateNode("red"), TruthValue(0.6, 0.9))
assign_property(ConceptNode("sparrow"), PredicateNode("red"), TruthValue(0.1, 0.9))
assign_property(ConceptNode("sparrow"), PredicateNode("small"), TruthValue(0.8, 0.9))
assign_property(ConceptNode("zebra"), PredicateNode("black"), TruthValue(0.5, 0.9))
assign_property(ConceptNode("zebra"), PredicateNode("small"), TruthValue(0.38, 0.9))

Define that all animals are actually animals

In [6]:
InheritanceLink(ConceptNode("frog"), ConceptNode("animal"))
InheritanceLink(ConceptNode("frog"), ConceptNode("animal"))
InheritanceLink(ConceptNode("tiger"), ConceptNode("animal"))
InheritanceLink(ConceptNode("tiger"), ConceptNode("animal"))
InheritanceLink(ConceptNode("sparrow"), ConceptNode("animal"))
InheritanceLink(ConceptNode("sparrow"), ConceptNode("animal"))
InheritanceLink(ConceptNode("zebra"), ConceptNode("animal"))
InheritanceLink(ConceptNode("zebra"), ConceptNode("animal"))

(InheritanceLink
  (ConceptNode "zebra") ; [1533366019804127650][1]
  (ConceptNode "animal") ; [4997559415121154116][1]
) ; [16824953705371388992][1]

**Query example**  
let's select all animals and attributes, filtering by some threshold


first define function and EvaluationLink

In [7]:
threshold = 0.6

def apply_threshold(atom):
    if atom.tv.mean < threshold:
        return TruthValue(0, 1)
    return TruthValue(1, 1)

Constrain Variable("X") is animal

In [8]:
is_animal = InheritanceLink(VariableNode("X"), ConceptNode("animal"))

EvaluationLink will accept link    X <- "animal" 
and return flag if it "matches"

In [9]:
predicate = EvaluationLink(VariableNode("$B"),
                           VariableNode("X"))

eval_threshold = EvaluationLink(
                    GroundedPredicateNode("py: apply_threshold"),
                    ListLink(predicate))

first define pattern that will be grounded during the search

In [10]:
and_link = AndLink(is_animal,
                   EvaluationLink(VariableNode("$B"),
                                 VariableNode("X")),
                   eval_threshold
                  )

### now build the BindLink
BindLink is constructed using pattern to ground and pattern to return,  
pattern to return should be part of pattern to ground

In [11]:
bindlink_filter = BindLink(and_link,
                           predicate)

execute query

In [12]:
print(execute_atom(atomspace, bindlink_filter))

(SetLink
  (EvaluationLink (stv 0.900000 0.900000)
    (PredicateNode "big")
    (ConceptNode "tiger")
  )
  (EvaluationLink (stv 0.800000 0.900000)
    (PredicateNode "small")
    (ConceptNode "sparrow")
  )
  (EvaluationLink (stv 0.600000 0.900000)
    (PredicateNode "red")
    (ConceptNode "tiger")
  )
  (EvaluationLink (stv 0.700000 0.900000)
    (PredicateNode "green")
    (ConceptNode "frog")
  )
  (EvaluationLink (stv 0.800000 0.900000)
    (PredicateNode "small")
    (ConceptNode "frog")
  )
)



## Exercise: use atom.tv.confidence

in threshold function